# Data Cleaning
This notebook will create a single dataframe to use in the rest of the analysis, using the 4 different data sources we have. 


## Setup

In [ ]:
import os
default_path='C:/Users/CAT/OneDrive - Equinor/Projects/Project Strech/Project v.1'
os.chdir(default_path)

import pandas as pd
import numpy as np
from functools import reduce
from functions import fill_month
from functions import id_diff
from functions import count_unique

Import the data sets

In [ ]:
#data showing who left the company
leav= pd.read_csv("data/raw/Leav_18.csv", sep=';')
#main dataframe
main= pd.read_csv("data/raw/full_data_monthly_18.csv", sep=';', encoding='latin-1')
#additional travel data
trv= pd.read_csv('data/raw/travel_costs_18.csv', sep=';', encoding='latin-1')
#additional timewriting data
absence= pd.read_csv('data/raw/time_absence_18.csv', sep=';', encoding='latin-1')
attend= pd.read_csv('data/raw/time_attendance_18.csv', sep=';', encoding='latin-1')

Import the dataset that has the country-nationality pairs to use as a dictionary later

In [ ]:
#df with country-nation pairs
count_dict= pd.read_csv('data/supporting/country dict.csv',
                        header=None,index_col=0,squeeze=True, encoding='latin-1').to_dict()

## Initial Cleanup

Rename the columns of the datasets to make sure all columns that have the matching data have the same name.

In [ ]:
#rename and clean columns
rename_dict= {'Cal. year / month': 'date' , 
             'Employee Equinor': 'id',
             'Employee Equinor.1':'name',
             'Length of service':'service_length',
             'Age in Years':'age',
             'Host Country':'host_country',
             'Nationality': 'nation',
             'Manager (ref. zansnr':'manager_id',
             'Manager (ref. zansnr.1':'manager',
             'Chief Position':'is_manager',
             'Orgunit BA':'BA',
             'Organizational level':'org_level',
             'Discipline.1':'discipline_id',
             'Discipline':'discipline_id',
             'Moves for Permanent employees (from)': 'Leavers', 
             'Branch of study':'study',
             'Process Network':'network', 
             'Certificate':'certif'
             }

leav = leav.rename(columns=rename_dict)
main=main.rename(columns=rename_dict)
attend=attend.rename(columns={'Cal. year / month': 'date' , 'Hours':'attend'})
absence=absence.rename(columns={'Cal. year / month': 'date' , 'Hours':'absence'})

Drop some rows that are created during data extraction and create lists of id's that we will use later. Lastly convert the numerical columns to numeric

In [ ]:
leav = leav[~leav.isin(['Result', 'Overall Result']).any(axis=1)]
main = main[~main.isin(['Result', 'Overall Result']).any(axis=1)]
absence = absence[~absence.isin(['Result', 'Overall Result']).any(axis=1)]
attend = attend[~attend.isin(['Result', 'Overall Result']).any(axis=1)]
trv = trv[~trv.isin(['Result', 'Overall Result']).any(axis=1)]

#drop a useless column 
main.drop(['Unnamed: 16'], axis=1, inplace=True)

#create a list of id's that were found in the main dataset 
id_list= list(main['id'].unique())
#create a list of id's that have left the company
leaver_list= list(leav['id'].unique())

#convert the numerical columns to numerical
#replace the , decimal with . and then round 
absence['absence'] = pd.to_numeric(absence['absence'].astype('str').apply(lambda x: x.replace(',', '.'))).round()
attend['attend'] = pd.to_numeric(attend['attend'].astype('str').apply(lambda x: x.replace(',', '.'))).round()
trv['travel_avg'] = pd.to_numeric(trv['travel_avg'].astype('str').apply(lambda x: x.replace(',', '.'))).round()

## Concenate

In [ ]:
#merge on id and name using outer to see if there are points from the dfs that don't match 
df=pd.DataFrame.merge(main,leav, on=['id', 'name'], how='outer')
df['Leavers']= df['Leavers'].replace(np.nan, 0) #If wasn't in the leavers dataset, then not leaver

#keep only the data where id includes the ids from the xtra info
df=df[df['id'].isin(id_list)] #this drops all the missing

All data sets have a date column but they have different structures. On top of that, there are different number of rows for each employee in each data set (see ReadMe). So we'll need to populate all the data sets so that every employee has one row per month in the data set. 

In [ ]:
#Complete missing month data
df = fill_month(df, '2017-12-01', '2018-12-31', 'MS') #data set is from 12.17 to 12.18
df.sort_values(by=['id', 'date'], inplace=True) 
df=df.groupby(['id']).ffill()
df=df.bfill()

absence= fill_month(absence, '2017-12-01', '2018-12-31', 'MS')
attend= fill_month(attend, '2017-12-01', '2018-12-31', 'MS')

attend['attend']= attend['attend'].replace(np.nan, 0) #If wasn't in the leavers dataset, then not leaver
absence['absence']= absence['absence'].replace(np.nan, 0) #If wasn't in the leavers dataset, then not leaver

absence=absence.drop(['name'], axis=1)
attend=attend.drop(['name'], axis=1)
trv=trv.drop(['name'], axis=1)

#same for the time and travel datasets
dfs=[df, absence, attend, trv]
for dfx in dfs:
    dfx['id']=pd.to_numeric(dfx['id'])

df1=pd.DataFrame.merge(df,trv, on=['id'], how='outer')

#merge multiple datasets
dfs=[df1, absence, attend]
df_final = reduce(lambda left,right: pd.merge(left,right,on=['id', 'date'], how='outer'), dfs)

#choose only the ids that were in the main dataset
df=df_final[df_final['id'].isin([int(x) for x in id_list])] 

df.isnull().sum() #the ids from time
#see if everyone has 14 rows
df.groupby('id')['id'].count().unique()


## Recoding Variables

In [ ]:
#___give numerical values to the band and org level 
band_dict={'PROF':3,'PRIN PROF':4,'LEAD PROF':5,'ASSOCIATE':2,'OPER & SUPP':1,
           'MANAGER':6, 'EXECUTIVE':7, 'SR EXEC':8}

org_lev_dict={'Dept level 5':6, 'Sector Level':5, 'Dept level 6':7, 'Business Unit':4,
       'Business Cluster':3, 'Business Area':2, 'Corporate':1, 'Dept level 7':8,
       'Dept level 8':9}

df['Band']= df['Band'].replace(band_dict)
df['org_level']= df['org_level'].replace(org_lev_dict)

#___fix the categorical variables to binary
df['is_manager']= df['is_manager'].replace({'#':0, 'X':1})
df['Gender']=df['Gender'].replace(2, 0)

## Handling Missing Values

### Converting different conotations for missing value to np.nan

In [ ]:
#count the number of Not assigned
for col in df.columns:
    if df[col].dtype == object:
        count = 0
        count = [count + 1 for x in df[col] if x == 'Not assigned']
        print(col + ' ' + str(sum(count)))

#convert different type of NaN to np.nan
df= df.replace('Not assigned',np.nan) 
df= df.replace('#',np.nan)

df.describe()
#If wasn't in the dataset, then replace the nan for that month with 0
df['attend']= df['attend'].replace(np.nan, 0) 
df['absence']= df['absence'].replace(np.nan, 0) 
df['travel_avg']= df['travel_avg'].replace(np.nan, 0) 
df['service_length']= df['service_length'].replace(np.nan, 0)     

#there are employees with different certificate information in different months
#replace the "meaningless" certificates with more meaningful certificate info if it exists
certif_dict = dict.fromkeys(['DD', 'CE', 'DE', 'BD', 'ED', 'BB', 'DG', 'EJ', 'DH', 'BC', 'BA',
       'BG', 'BH', 'EE', 'EC', 'CB', 'DF',  'DI', 'CA'], np.nan)
certif_dict['99']=np.nan 
df['certif']= df['certif'].replace(certif_dict)
df.sort_values(by=['id', 'date'], inplace=True) 
df[['id', 'certif']]=df[['id', 'certif']].groupby(['id']).ffill()
df[['id','certif']]=df[['id', 'certif']].groupby(['id']).bfill()

#convert numeric to numeric
cont_var=['service_length', 'manager_id', 'certif'] #numeric variables
for x in cont_var:
    df[x]=pd.to_numeric(df[x], errors='raise')

#dataframe with columns that have missing values
miss=df[df.isnull().any(axis=1)]
#see if there are employees that have only some months missing within a column
miss['Leavers'].sum()/13

### Fill some missing values

In [ ]:
#fill missing org_level based on the average org_level of the same band
miss_ids= list(df['id'][df.isnull().sum(axis=1)>4].unique())
df=df[~df['id'].isin(miss_ids)] 

#Fill missing Bands and org level based on the averages
df['org_level'].fillna(df.groupby('Band')['org_level'].transform('mean').round(), inplace=True)
#Band: assign average band level aggregataed over org level and age
df['Band'].fillna(df.groupby(['org_level', 'Age'])['Band'].transform('mean').round(), inplace=True)
df['certif'].fillna(df.groupby(['Band', 'is_manager'])['certif'].transform('median').round(), inplace=True)

df.drop(columns='manager', inplace=True)

df.sort_values(by=['id', 'date'], inplace=True) 
df[['id', 'manager_id']]=df[['id', 'manager_id']].groupby(['id']).ffill()
df[['id','manager_id']]=df[['id', 'manager_id']].groupby(['id']).bfill()

#assing a manager to missing managers based on most common manager within band, ba and org_level combo
df['manager_id'].fillna(df.groupby(['Band', 'BA', 'org_level'])['manager_id'].transform('median'), inplace=True)

#check the list of employees who still have a missing value
miss_ids= list(df['id'][df.isnull().sum(axis=1)>0].unique())
df=df[~df['id'].isin(miss_ids)] 
df.isnull().sum()

## Feature Engineering

In [ ]:
#create historical data for host country and discipline
#count number of countries/disciplines each person have changes in the last year
df['host_count']=df.groupby('id')['host_country'].transform('nunique')
df['network_count']=df.groupby('id')['network'].transform('nunique')

#create promotion data
df.sort_values(by=['id', 'date'], inplace=True) 
df['promotion_1'] = df['Band'].diff() #create new variable with the difference of promotion variable latter row-previous row
mask = df.id != df.id.shift(1) #create mask where ids are different from row to row (new person) 
df['promotion_1'][mask] = 0 
#copy the data to all rows promotion column for each person
df['promotion'] = df.groupby(['id'])['promotion_1'].apply(lambda x: x.cumsum())
df.drop(columns=['promotion_1'], inplace=True)
#df.isnull().sum()

#create a new variable called expat based on the nation and host country information
df['nation'].head()
df['host_country'].head()
#all lower case
df.nation = df.nation.astype(str).apply(lambda x: x.lower())
df.host_country = df.host_country.astype(str).apply(lambda x: x.lower())
count_dict= {k.lower(): v.lower() for k, v in count_dict.items()}

df['nation']= df['nation'].replace({'nan':np.nan,'kasachstani':'kazakh',
  'columbian':'colombian', 'brithish':'british','white-russian':'belarusian',
  'f. trinidad & t':'trinidadian', 'ghanian':'ghanaian', 'neth.antillean':'antillean'})

df['host_country']= df['host_country'].replace({'nan':np.nan,'brasil':'brazil','great britain': 'united kingdom', 'russian fed.': 'russia'})
#create the home country column
df['home_country']= df['nation'].map(count_dict)
df['home_country'].head()

#create the expat columns
df.loc[df['home_country']!=df['host_country'], 'expat'] = 1
df.loc[df['home_country']==df['host_country'], 'expat'] = 0
#change some mismatch between home-host country that's caused by nan into nans 
df['expat'] =np.where(((df['host_country'].isnull()) | (df['home_country'].isnull())), np.nan, df['expat'] )

#create team size
#team size of people, based on shared manager, so managers are only counted in the team of their manager
df['team_size']= df.groupby('manager_id')['id'].transform('nunique') #this wont be accurate because of filling the missing values

df['is_norsk']= np.where(df['nation']=='norwegian', 1, 0)
df['in_norge']= np.where(df['host_country']=='norway', 1, 0)

df['attend_avg']=df.groupby(['id'])['attend'].transform('mean').round()
df['absence_avg']=df.groupby(['id'])['absence'].transform('mean').round()

#see the minimum row number someone has
df.groupby('id')['id'].count().unique()
miss=df[df.isnull().any(axis=1)]


## Single row per person
So far we had 13 rows per person, one row per month. But running analysis with that data is like using duplicates of your data with small adjustments. So I kept the data only for the last month for each person. 

Before I did that, I created a new column per column to add the info about each person's manager. So for each employee manager age, manager gender, manager certificate etc. 

In [ ]:
#list of managers
manager_ids=list(df['manager_id'].unique())
#dataframe for the managers
managers=df[df['id'].isin(manager_ids)]

#use the managers dataset to get info about each persons manager
df_man = pd.merge(df, managers.set_index(['date','id']), left_on=['date', 'manager_id'], how='left', right_index=True, suffixes=('', '_man'))

df_man.dropna(inplace=True)

#narrow the dataset to have single line per person
idx = df_man.groupby('id')['date'].transform(max) == df_man['date'] #choose the last date for each person
df_narrow = df_man[idx]




# Save the dataset

In [ ]:
df_narrow.to_csv('data/processed/single_df_filled.csv', encoding='utf-8', index=False)
df_man.to_csv('data/processed/full_df_filled.csv', encoding='utf-8', index=False)